# 64-Band Real-Time Graphical Equalizer
This notebook records audio from the microphone, applies a 64-band graphical equalizer in real-time, and plays it back using sounddevice.

In [ ]:
!pip install sounddevice numpy scipy ipywidgets

In [ ]:
import numpy as np
import sounddevice as sd
from scipy.signal import butter, sosfilt
import ipywidgets as widgets
from IPython.display import display
import threading

# Audio parameters
fs = 44100
blocksize = 1024
channels = 1

# Frequency bands (log spaced)
freqs = np.logspace(np.log10(20), np.log10(20000), 65)

# Create sliders
sliders = []
for i in range(64):
    slider = widgets.FloatSlider(
        value=0.0,
        min=-12.0,
        max=12.0,
        step=0.5,
        description=f'{int(freqs[i])}Hz',
        orientation='vertical',
        layout=widgets.Layout(height='200px', width='30px')
    )
    sliders.append(slider)

display(widgets.HBox(sliders))

# Design band filters
sos_filters = []
for i in range(64):
    low = freqs[i] / (fs/2)
    high = freqs[i+1] / (fs/2)
    sos = butter(2, [low, high], btype='band', output='sos')
    sos_filters.append(sos)

# Filter states
states = [np.zeros((sos.shape[0], 2)) for sos in sos_filters]

def audio_callback(indata, outdata, frames, time, status):
    global states
    x = indata[:, 0]
    y = np.zeros_like(x)
    
    for i in range(64):
        gain = 10**(sliders[i].value / 20)
        filtered = sosfilt(sos_filters[i], x, zi=states[i])[0]
        y += gain * filtered
    
    outdata[:, 0] = y

stream = sd.Stream(
    samplerate=fs,
    blocksize=blocksize,
    channels=channels,
    callback=audio_callback
)

def start_audio():
    stream.start()

def stop_audio():
    stream.stop()

start_button = widgets.Button(description='Start')
stop_button = widgets.Button(description='Stop')

start_button.on_click(lambda x: start_audio())
stop_button.on_click(lambda x: stop_audio())

display(widgets.HBox([start_button, stop_button]))